In [1]:
!pip -q install openpyxl xlrd

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/forecast-homelessness/2007-2020-HIC-Counts-by-CoC.xlsx
/kaggle/input/forecast-homelessness/2007-2020-PIT-Estimates-by-CoC.xlsx
/kaggle/input/forecast-homelessness/2007-2020-PIT-Estimates-by-state.xlsx
/kaggle/input/forecast-homelessness/2007-2020-HIC-Counts-by-State.xlsx
/kaggle/input/forecast-homelessness/2011-2020-PIT-Veteran-Counts-by-State.xlsx
/kaggle/input/forecast-homelessness/2011-2020-PIT-Veteran-Counts-by-CoC.xlsx


### collect yearly rent data of us states

In [3]:
def get_text_in_parenthesis(s):
    import re
    return re.findall('\(.*?\)', s)[0].strip("()")

In [4]:
from bs4 import BeautifulSoup
import requests
base_url = "https://www.rentdata.org/states"

rent_data = pd.DataFrame()

for yr in range(2006, 2021):
    yr_url = f"{base_url}/{yr}"
    html = requests.get(yr_url).text
    soup = BeautifulSoup(html, "lxml")
    all_table = soup.find("table", attrs={"id": "state_table"})
    tt = pd.read_html(str(all_table))
    yr_df = tt[0]
    yr_df['Date'] = f"31-01-{yr}"
    rent_data = rent_data.append(yr_df)
    
rent_data = rent_data.rename(columns={'State':"State_Name"})
rent_data['State'] = rent_data['State_Name'].apply(get_text_in_parenthesis)
rent_data['Date'] = pd.to_datetime(rent_data['Date'])
rent_data.shape, rent_data.columns, rent_data['State'].nunique()

((820, 9),
 Index(['State_Name', '0 BR', '1 BR', '2 BR', '3 BR', '4 BR', 'Est. Population',
        'Date', 'State'],
       dtype='object'),
 56)

In [5]:
for col in ['0 BR', '1 BR', '2 BR', '3 BR', '4 BR',]:
    rent_data[col] = rent_data[col].str.replace("$","")
    rent_data[col] = rent_data[col].str.replace(",","")
    rent_data[col] = rent_data[col].str.replace(" ","").astype(float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [6]:
rents_df = pd.DataFrame()
for state in rent_data['State'].unique():
    state_df = rent_data[rent_data['State'].isin([state])]
    state_df = state_df[['Date', '0 BR']]
    upsample = state_df.set_index(['Date']).resample('M')
    interpolated = upsample.interpolate(method='time').reset_index()
    interpolated['State'] = state
    rents_df = rents_df.append(interpolated.round())
    
rents_df = rents_df.rename(columns={'Year':'Date', '0 BR': 'Min_Rent'})
rents_df.shape, rents_df.columns

((9224, 3), Index(['Date', 'Min_Rent', 'State'], dtype='object'))

### homelessness data

In [7]:
from openpyxl import load_workbook

working_dir = "/kaggle/input/forecast-homelessness"
pit_path = f"{working_dir}/2007-2020-PIT-Estimates-by-state.xlsx"
hit_path = f"{working_dir}/2007-2020-HIC-Counts-by-State.xlsx"

def get_sheetnames_xlsx(filepath):
    wb = load_workbook(filepath, read_only=True, keep_links=False)
    return wb.sheetnames

sheet_names = get_sheetnames_xlsx(hit_path)
print(sheet_names)

['2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', 'Revisions']


In [8]:
consd_df = pd.DataFrame()
for year in range(2007, 2021):
    df = pd.read_excel(pit_path, sheet_name=str(year))
    df['Year'] = f"{year}-01-31"
    # tdf = df[df['State'].isin(['TX', "Texas", "texas"])]
    # print(tdf.shape)
    df.columns = [c.strip().strip(f'{year}').strip(" ,") 
                  for c in df.columns]
    consd_df = consd_df.append(df)
    

nans={"": np.nan,' ':np.nan, 'NA':np.nan,'N/A':np.nan, 'NaN':np.nan}
consd_df = consd_df.replace(nans)
consd_df = consd_df[~consd_df['State'].isin(['Total','total'])]
consd_df = consd_df.dropna(subset=['Overall Homeless', 'State'])
consd_df = consd_df[['State', 'Year', 'Overall Homeless']].reset_index(drop=1)
consd_df['Year'] = pd.to_datetime(consd_df['Year'])
consd_df['Overall Homeless'] = consd_df['Overall Homeless'].astype(int)
consd_df.shape, consd_df.columns

((760, 3), Index(['State', 'Year', 'Overall Homeless'], dtype='object'))

In [9]:
consd_df.isna().sum()

State               0
Year                0
Overall Homeless    0
dtype: int64

In [10]:
pit_df = pd.DataFrame()
for state in consd_df['State'].unique():
    state_df = consd_df[consd_df['State'].isin([state])]
    state_df = state_df.drop(columns=['State'])
    upsample = state_df.set_index(['Year']).resample('M')
    interpolated = upsample.interpolate(method='time').reset_index()
    interpolated['State'] = state
    pit_df = pit_df.append(interpolated.round())
    
pit_df = pit_df.rename(columns={'Year':'Date'})
pit_df.shape, pit_df.columns

((8515, 3), Index(['Date', 'Overall Homeless', 'State'], dtype='object'))

In [11]:
# interpolated = upsampled.interpolate(method='spline', order=2)

### installing selenium to collect labour stats data

In [12]:
# Setting-up Environment & Importing necessary libraries
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
from bs4 import BeautifulSoup
from shutil import make_archive
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

# Setting-up web driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

     |████████████████████████████████| 954 kB 4.5 MB/s 
     |████████████████████████████████| 356 kB 66.4 MB/s 
Get:1 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5388 B]
Get:2 http://packages.cloud.google.com/apt cloud-sdk-bionic InRelease [6786 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://packages.cloud.google.com/apt gcsfuse-bionic/main amd64 Packages [531 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://packages.cloud.google.com/apt cloud-sdk-bionic/main amd64 Packages [214 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2398 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1434 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [632 kB]
Get

In [13]:
# id="latest-numbers"
from bs4 import BeautifulSoup
import requests

base_url = "https://www.bls.gov"
url="https://www.bls.gov/lau/#tables"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text
# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
all_table = soup.find("div", attrs={"id": "latest-numbers"})
all_ptags = all_table.find_all('p')

In [14]:
import time
timeout = 30
consd_emp_df = pd.DataFrame()

i = 0
while i < len(all_ptags):
    ptag = all_ptags[i]
    url_part = ptag.find('a')['href']
    full_url = f"{base_url}{url_part}"
    driver.get(full_url)
    state_splits = [t for t in ptag.text.split()[:2] if t.istitle()]
    state = " ".join(state_splits).replace(".", '')
    try:
        year_ec = EC.element_to_be_clickable((By.XPATH,
                    "//select[@id='from-year']/option[text()='2000']"))
        from_year = WebDriverWait(driver, timeout).until(year_ec)
        from_year.click()
        time.sleep(3)
        go_ec = EC.element_to_be_clickable((By.XPATH,"//input[@alt='Retrieve Data Now!']"))
        go_but = WebDriverWait(driver, timeout).until(go_ec)
        go_but.click()
        print("success:", state)
    except:
        print("failed:", state)
        continue
        
    time.sleep(3)
    i += 1
    go_soup = BeautifulSoup(driver.page_source, "html.parser")
    tables = go_soup.find('table', attrs={"id": "table0"})
    temp_df = pd.read_html(str(tables))[0]
    temp_df['State_Name'] = state
    consd_emp_df = consd_emp_df.append(temp_df)
    # print(ptag.text.split()[0], consd_emp_df.shape)
    # driver.current_url

consd_emp_df.shape, consd_emp_df.columns

success: Alabama
success: Alaska
success: Arizona
failed: Arkansas
success: Arkansas
failed: California
success: California
success: Colorado
success: Connecticut
success: Delaware
success: DC
success: Florida
success: Georgia
success: Hawaii
success: Idaho
success: Illinois
success: Indiana
success: Iowa
success: Kansas
success: Kentucky
success: Louisiana
failed: Maine
failed: Maine
success: Maine
success: Maryland
success: Massachusetts
success: Michigan
success: Minnesota
success: Mississippi
success: Missouri
success: Montana
success: Nebraska
success: Nevada
success: New Hampshire
failed: New Jersey
success: New Jersey
success: New Mexico
success: New York
success: North Carolina
success: North Dakota
success: Ohio
success: Oklahoma
success: Oregon
success: Pennsylvania
success: Puerto Rico
failed: Rhode Island
success: Rhode Island
success: South Carolina
success: South Dakota
success: Tennessee
success: Texas
failed: Utah
success: Utah
success: Vermont
success: Virginia
success

((13624, 9),
 Index(['Year', 'Period', 'labor force participation rate',
        'employment-population ratio', 'labor force', 'employment',
        'unemployment', 'unemployment rate', 'State_Name'],
       dtype='object'))

### US states 2 letter abbrevations

In [15]:
abbv_url = "https://www.ssa.gov/international/coc-docs/states.html"
# Make a GET request to fetch the raw HTML content
abv_html = requests.get(abbv_url).text
# Parse the html content
abbv_soup = BeautifulSoup(abv_html, "lxml")
table = abbv_soup.find("table")

abb_df = pd.read_html(str(table))[0]
abb_df.columns = ['State_Name','Shortform']

abb_df['State_Name'] = abb_df['State_Name'].str.strip()
abb_df['State_Name'] = abb_df['State_Name'].str.lower()
abb_map = dict(zip(abb_df['State_Name'], abb_df['Shortform']))

In [16]:
emp_df = consd_emp_df.copy()
emp_df['State_Abb'] = emp_df['State_Name'].str.strip()
emp_df['State_Abb'] = emp_df['State_Abb'].str.lower()
emp_df['State_Abb'] = emp_df['State_Abb'].replace(abb_map)

In [17]:
month_map = dict(zip(pd.date_range('2020-01-01', 
            freq='M', periods=12).strftime('%b'),range(1,13)))
emp_df = emp_df[emp_df['Period'].isin(month_map.keys())].reset_index(drop=1)
emp_df['Month'] = emp_df['Period'].replace(month_map)
emp_df['Date'] = (emp_df['Year'].astype(str) + '-' 
                        + emp_df['Month'].astype(str))
emp_df['Date'] = pd.to_datetime(emp_df['Date'], 
               format="%Y-%m") + pd.offsets.MonthEnd(0)

### collect census data from its API

In [18]:
import requests
import ast

HOST = "https://api.census.gov/data/timeseries/poverty/saipe"
GET = "?get=NAME,SAEMHI_PT,SAEPOVALL_PT,SAEPOVRTALL_PT"

# SAEMHI_PT -  median house hold income estimate
# SAEPOVRTALL_PT - poverty rate estimate
# SAEPOVALL_PT - poverty count

state_range = [str(n).zfill(2) for n in range(1, 57)]
year_range = range(2000, 2020)

data_rows = []
for state in state_range:
    for year in year_range:
        PARAM = f"&for=state:{state}&YEAR={year}"
        URL = HOST + GET + PARAM
        r = requests.get(URL)
        # print(state, year)
        try:
            row = r.json()[-1]
        except:
            row = [np.nan, np.nan, np.nan, 
                   np.nan, year, state]
        data_rows.append(row)

In [19]:
col_names = ['State_Name', 'MHHI', 
             'Poverty_Count', 'Poverty_Rate',
             "Year", 'State_Num']
censes_df = pd.DataFrame(data_rows, columns=col_names)
censes_df['Year'] = censes_df['Year'].apply(lambda x: f"{x}-01-31")
censes_df['Year'] = pd.to_datetime(censes_df['Year'])
censes_df = censes_df.dropna(subset=['Year', 'State_Name'])
censes_df = censes_df.drop_duplicates(subset=['Year','State_Name'])
censes_df['MHHI'] = censes_df['MHHI'].astype(float)
censes_df['Poverty_Count'] = censes_df['Poverty_Count'].astype(float)
censes_df['Poverty_Rate'] = censes_df['Poverty_Rate'].astype(float)
censes_df.shape, censes_df.columns

((1020, 6),
 Index(['State_Name', 'MHHI', 'Poverty_Count', 'Poverty_Rate', 'Year',
        'State_Num'],
       dtype='object'))

In [20]:
# state_df.set_index(['Year']).resample('M').asfreq()

In [21]:
cdf = pd.DataFrame()
for state in censes_df['State_Name'].unique():
    state_df = censes_df[censes_df['State_Name'].isin([state])]
    state_df = state_df.drop(columns=['State_Name', 'State_Num'])
    upsample = state_df.set_index(['Year']).resample('M')
    interpolated = upsample.interpolate(method='values').reset_index()
    interpolated['State'] = state
    cdf = cdf.append(interpolated.round())
    
cdf = cdf.rename(columns={'Year':'Date'})
cdf['State'] = cdf['State'].str.strip()
cdf['State'] = cdf['State'].str.lower()
cdf['State'] = cdf['State'].replace(abb_map)
cdf.shape, cdf.columns

((11679, 5),
 Index(['Date', 'MHHI', 'Poverty_Count', 'Poverty_Rate', 'State'], dtype='object'))

### Zillow data

In [22]:
zillow_link = ("https://files.zillowstatic.com/research/public_csvs" + 
               "/zhvi/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1635180214")
response = requests.get(zillow_link)
with open('zillow_house_prices.csv', 'wb') as f:
    f.write(response.content)

In [23]:
housing_raw = pd.read_csv("zillow_house_prices.csv")
housing_raw = housing_raw.drop(columns=['RegionID','SizeRank','RegionType'])
housing_raw = housing_raw.dropna(subset=['StateName'])
housing_raw.shape, housing_raw.columns

((908, 263),
 Index(['RegionName', 'StateName', '2000-01-31', '2000-02-29', '2000-03-31',
        '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
        ...
        '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
        '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30'],
       dtype='object', length=263))

In [24]:
hdf = housing_raw.melt(['StateName', 'RegionName'], 
                       var_name='Date', value_name='HomeValueIndex')

hdf = hdf.groupby(['StateName', 'Date'])['HomeValueIndex'].median().reset_index()
hdf['Date'] = pd.to_datetime(hdf['Date'])
hdf = hdf.rename(columns={'StateName': 'State'})
hdf.shape, hdf.columns

((13311, 3), Index(['State', 'Date', 'HomeValueIndex'], dtype='object'))

In [25]:
emp_df = emp_df.rename(columns={'unemployment rate': 'unemploy_rate',
                        'employment-population ratio': 'employed_pop_rate',
                                'State_Abb': 'State'})
emp_df = emp_df[['State_Name', 'State', 'Date',
                 'unemploy_rate', 'employed_pop_rate']]


emp_df['unemploy_rate'] = (emp_df['unemploy_rate'].str
                           .extract(r'(\d+.\d+)').astype('float'))
emp_df['employed_pop_rate'] = (emp_df['employed_pop_rate'].str
                               .extract(r'(\d+.\d+)').astype('float'))

# upper case the abbrevations
pit_df['State'] = pit_df['State'].str.upper()
emp_df['State'] = emp_df['State'].str.upper()
hdf['State'] = hdf['State'].str.upper()
cdf['State'] = cdf['State'].str.upper()


# filter on the common states
common_states = (set(pit_df['State']) & set(emp_df['State']) 
                 & set(hdf['State']) & set(cdf['State']))
pit_df = pit_df[pit_df['State'].isin(common_states)]
emp_df = emp_df[emp_df['State'].isin(common_states)]
hdf = hdf[hdf['State'].isin(common_states)]
cdf = cdf[cdf['State'].isin(common_states)]
rents_df = rents_df[rents_df['State'].isin(common_states)]

pit_df.columns, emp_df.columns, hdf.columns

(Index(['Date', 'Overall Homeless', 'State'], dtype='object'),
 Index(['State_Name', 'State', 'Date', 'unemploy_rate', 'employed_pop_rate'], dtype='object'),
 Index(['State', 'Date', 'HomeValueIndex'], dtype='object'))

### Joining all the data based on datetime (few datasets are interpolated form yearly to monthly)

In [26]:
data_df = pit_df.merge(emp_df, how="left", on=['State', 'Date'])
data_df = data_df.merge(hdf, how="left", on=['State', 'Date'])
data_df = data_df.merge(cdf, how="left", on=['State', 'Date'])
data_df = data_df.merge(rents_df, how="left", on=['State', 'Date'])
data_df.shape, data_df.columns

((8007, 11),
 Index(['Date', 'Overall Homeless', 'State', 'State_Name', 'unemploy_rate',
        'employed_pop_rate', 'HomeValueIndex', 'MHHI', 'Poverty_Count',
        'Poverty_Rate', 'Min_Rent'],
       dtype='object'))

In [27]:
data_df[data_df['State'].isin(['TX'])].isna().sum()

Date                  0
Overall Homeless      0
State                 0
State_Name            0
unemploy_rate         0
employed_pop_rate     0
HomeValueIndex        0
MHHI                 12
Poverty_Count        12
Poverty_Rate         12
Min_Rent              0
dtype: int64

In [28]:
print("Unique States covered:", data_df['State'].nunique())
data_df.isna().sum()

Unique States covered: 51


Date                   0
Overall Homeless       0
State                  0
State_Name             0
unemploy_rate          0
employed_pop_rate      0
HomeValueIndex        24
MHHI                 612
Poverty_Count        612
Poverty_Rate         612
Min_Rent               0
dtype: int64

In [29]:
emp_df.to_csv("yearly_employment_rates.csv", index=False)
abb_df.to_csv("state_abbrevations.csv", index=False)
pit_df.to_csv("pit_data_interpolated.csv", index=False)
hdf.to_csv("zillow_home_price_index.csv", index=False)
cdf.to_csv("census_poverty_income.csv", index=False)
rents_df.to_csv("yearly_house_rents.csv", index=False)
data_df.to_csv("unified_data.csv", index=False)

In [30]:
from IPython.display import FileLinks
FileLinks(".")

./
  pit_data_interpolated.csv
  state_abbrevations.csv
  zillow_home_price_index.csv
  unified_data.csv
  yearly_employment_rates.csv
  census_poverty_income.csv
  yearly_house_rents.csv
  __notebook__.ipynb
  zillow_house_prices.csv